In [ ]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Hawaii",
                },
            },
            "required": ["pizza_name"],
        },
    },
    {
        "name": "place_order",
        "description": "Place an order for a pizza from the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza you want to order, e.g. Margherita",
                },
                "quantity": {
                    "type": "integer",
                    "description": "The number of pizzas you want to order",
                    "minimum": 1
                },
                "address": {
                    "type": "string",
                    "description": "The address where the pizza should be delivered",
                },
            },
            "required": ["pizza_name", "quantity", "address"],
        },
    }
]


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain

In [ ]:

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """You are an AI chatbot having a conversation with a human.

Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["human_input"], template=template)

chain = create_openai_fn_chain(functions, llm, prompt, verbose=True)

In [ ]:
chain.run("How much does Pizza hawaii cost?")

In [ ]:
chain.run("I want to order two pizza hawaii to 123 Fakestreet")

In [ ]:
# chain.run("How old did the queen get?") # thats so bad!

### How does it work with the vanilla API?

In [ ]:
import openai

def chat(query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query}],
        functions=functions
    )
    message = response["choices"][0]["message"]
    return message

In [ ]:
chat("I want to order two pizza hawaii to 123 Fakestreet")

In [ ]:
chat("How old did the queen get?")

In [ ]:
message = chat("I want to order two pizza hawaii to 123 Fakestreet")
message

In [ ]:
print(type(message))

if message.get("function_call"): # not a real dictionary
    print("function call needed!")

### Lets create a fake database

In [ ]:
fake_db = {
    "pizzas": {
        "Hawaii": {"price": 15.00, "ingredients": ["ham", "pineapple", "cheese"]},
        "Margherita": {"price": 10.00, "ingredients": ["tomato", "mozzarella", "basil"]},
        "Pepperoni": {"price": 12.50, "ingredients": ["pepperoni", "mozzarella", "tomato sauce"]},
        "Veggie": {"price": 11.00, "ingredients": ["bell peppers", "onions", "olives", "mushrooms"]},
    },
    "orders": []
}

In [ ]:
def get_pizza_info(pizza_name):
    pizza = fake_db["pizzas"].get(pizza_name)
    
    if not pizza:
        return f"No information available for pizza: {pizza_name}"

    return {"name": pizza_name, "price": pizza["price"], "ingredients": pizza["ingredients"]}

def place_order(pizza_name, quantity, address):
    if pizza_name not in fake_db["pizzas"]:
        return f"We don't have {pizza_name} pizza!"
    
    if quantity < 1:
        return "You must order at least one pizza."
    
    order_id = len(fake_db["orders"]) + 1
    order = {
        "order_id": order_id,
        "pizza_name": pizza_name,
        "quantity": quantity,
        "address": address,
        "total_price": fake_db["pizzas"][pizza_name]["price"] * quantity
    }

    fake_db["orders"].append(order)
    
    return f"Order placed successfully! Your order ID is {order_id}. Total price is ${order['total_price']}."


In [ ]:
message = chat("I want to order two pizza hawaii to 123 Fakestreet")
message

In [ ]:
import json

print(message["function_call"]["name"])
print(json.loads(message["function_call"]["arguments"]))


In [ ]:
function_name = message["function_call"]["name"]
arguments = json.loads(message["function_call"]["arguments"])

response = place_order(**arguments)
print(response)

### Better - Let´s make that dynamic

In [ ]:
import openai
import json

class ChatBot:
    
    def __init__(self, database):
        self.fake_db = database
        
    def chat(self, query):
        initial_response = self.make_openai_request(query)
        
        message = initial_response["choices"][0]["message"]
        
        if message.get("function_call"):
            function_name = message["function_call"]["name"]
            arguments = json.loads(message["function_call"]["arguments"])
            
            function_response = getattr(self, function_name)(**arguments)
            
            follow_up_response = self.make_follow_up_request(query, message, function_name, function_response)
            return follow_up_response["choices"][0]["message"]["content"]
        else:
            return message["content"]
    
    def make_openai_request(self, query):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "user", "content": query}],
            functions=functions
        )
        return response

    def make_follow_up_request(self, query, initial_message, function_name, function_response):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": query},
                initial_message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return response

    def place_order(self, pizza_name, quantity, address):
        if pizza_name not in self.fake_db["pizzas"]:
            return f"We don't have {pizza_name} pizza!"
        
        if quantity < 1:
            return "You must order at least one pizza."
        
        order_id = len(self.fake_db["orders"]) + 1
        order = {
            "order_id": order_id,
            "pizza_name": pizza_name,
            "quantity": quantity,
            "address": address,
            "total_price": self.fake_db["pizzas"][pizza_name]["price"] * quantity
        }

        self.fake_db["orders"].append(order)
        
        return f"Order placed successfully! Your order ID is {order_id}. Total price is ${order['total_price']}."

    def get_pizza_info(self, pizza_name):
        if pizza_name in self.fake_db["pizzas"]:
            pizza = self.fake_db["pizzas"][pizza_name]
            return f"Pizza: {pizza['name']}, Price: ${pizza['price']}"
        else:
            return f"We don't have information about {pizza_name} pizza."

database = {
    "pizzas": {
        "Hawaii": {
            "name": "Hawaii",
            "price": 15.0
        },
        "Margherita": {
            "name": "Margherita",
            "price": 12.0
        }
    },
    "orders": []
}


In [ ]:
bot = ChatBot(database=database)

bot.chat("I want to order one pizza Margherita to 124 Fakestreet")
response = bot.chat("I want to order two pizza hawaii to 123 Fakestreet")

In [ ]:
response

In [ ]:
response= bot.chat("I want to order one pizza Salami to 124 Fakestreet")
response

In [ ]:
print(database)